In [16]:
import h5py
import keras
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

In [17]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))
    return x_data, y_data

clean_data_file = './data/cl/valid.h5'
poisoned_data_file = './data/bd/bd_valid.h5'
model_file = './model/bd_net.h5'

clean_x_test, clean_y_test = data_loader(clean_data_file)
bad_x_test, bad_y_test = data_loader(poisoned_data_file)
bad_model = keras.models.load_model(model_file)

clean_label_p = np.argmax(bad_model.predict(clean_x_test), axis=1)
clean_classification_accuracy = np.mean(np.equal(clean_label_p, clean_y_test))*100
print('Clean Classification accuracy:', clean_classification_accuracy)

bad_label_p = np.argmax(bad_model.predict(bad_x_test), axis=1)
attack_success_rate = np.mean(np.equal(bad_label_p, bad_y_test))*100
print('Attack Success Rate:', attack_success_rate)

361/361 [==============================] - 7s 19ms/step
Clean Classification accuracy: 98.64899974019225
361/361 [==============================] - 7s 19ms/step
Attack Success Rate: 100.0


### Visualizing data

In [ ]:
rows, cols = 3, 3
fig, axes = plt.subplots(rows, cols, figsize=(5, 4))
axes = axes.flatten()

for i in range(rows * cols):
    ax = axes[i]
    ax.imshow(clean_x_test[i]/255)
    ax.set_title(f"Label: {clean_y_test[i]}", fontsize = 10)
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
rows, cols = 3, 3
fig, axes = plt.subplots(rows, cols, figsize=(5, 4))
axes = axes.flatten()

for i in range(rows * cols):
    ax = axes[i]
    ax.imshow(bad_x_test[i]/255)
    ax.set_title(f"Label: {bad_y_test[i]}", fontsize = 10)
    ax.axis("off")

plt.tight_layout()
plt.show()

### Pruning Defense
The basic concept involves trimming the neural network and evaluating its performance in comparison to the original network to identify any discrepancies caused by the existence of a backdoor. It is important to note that backdoors activate unused or redundant neurons in the network.

The pruning defense operates in the following manner: the defender applies the received deep neural network (DNN) from the attacker to clean inputs from the validation dataset, Dvalid, and records the average activation of each neuron. Subsequently, the defender systematically prunes neurons from the DNN in increasing order of average activations, while noting the accuracy of the pruned network at each iteration. The defense concludes when the accuracy on the validation dataset falls below a predetermined threshold.

In [ ]:
model_weights = './model/bd_weights.h5'
bad_model.load_weights(model_weights)

cloned_model= keras.models.clone_model(bad_model)
cloned_model.set_weights(bad_model.get_weights())

'sorted_channels' contains the indices of the channels of the 'pool_3' layer sorted in increasing order of their mean activation values

In [ ]:
clean_accuracy=98.64899974019225

pooling_layer = cloned_model.get_layer('pool_3').output
intermediate_model = keras.models.Model(inputs=cloned_model.input, outputs=pooling_layer) #create a new intermediate model that takes same input as
#cloned_model but has output as pooling layer
intermediate_predictions = intermediate_model.predict(clean_x_test)

# get average activation values
average_activations = np.mean(intermediate_predictions, axis=(0, 1, 2))
sorted_channels = np.argsort(average_activations)

pruned_clean_accuracies = []
attack_success_rates = []
pruning_degree=[]
saved_models = np.zeros(3, dtype=bool)
pruned_weights = cloned_model.layers[5].get_weights()[0]
pruned_biases = cloned_model.layers[5].get_weights()[1]

for pruned_channel_index in tqdm(sorted_channels):
    # Prune the channel
    # pruned_weights, pruned_biases = cloned_model.get_layer('pool_3').get_weights()
    pruned_weights[:, :, :, pruned_channel_index] = 0
    pruned_biases[pruned_channel_index] = 0
    cloned_model.layers[5].set_weights([pruned_weights, pruned_biases])

    # Evaluate clean accuracy
    pruned_clean_predictions = cloned_model.predict(clean_x_test)
    pruned_clean_labels = np.argmax(pruned_clean_predictions, axis=1)
    pruned_clean_accuracy = np.mean(np.equal(pruned_clean_labels, clean_y_test)) * 100

    # Check for accuracy drop and save the model
    for i, accuracy_drop_threshold in enumerate([2, 4, 10]):
        if (clean_accuracy - pruned_clean_accuracy >= accuracy_drop_threshold and not saved_models[i]):
            print(f"The accuracy drops at least {accuracy_drop_threshold}%, saved the model")
            cloned_model.save(f'model_X={accuracy_drop_threshold}.h5')
            saved_models[i] = True

    # Append results
    pruned_clean_accuracies.append(pruned_clean_accuracy)
    attack_success_rate = np.mean(np.equal(np.argmax(cloned_model.predict(bad_x_test), axis=1), bad_y_test)) * 100
    attack_success_rates.append(attack_success_rate)
    pruning_degree.append(pruned_channel_index)

    # Print results
    print()
    print("The pruned clean accuracy is: ", pruned_clean_accuracy)
    print("The attack success rate is: ", attack_success_rate)
    print("The pruned channel index is: ", pruned_channel_index)

    # Clear the Keras session
    keras.backend.clear_session()

In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(pruning_degree, pruned_clean_accuracies, marker='o', label='Clean Data Accuracies')
plt.plot(pruning_degree, attack_success_rates, marker='x', color='red', label='Attack Success Rates')

plt.title('Clean Data Accuracies and Attack Success Rates vs. Degree of Pruning')
plt.xlabel('Degree of Pruning')
plt.ylabel('Percentage')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
x_axis = np.arange(1,61)/60
plt.plot(x_axis,pruned_clean_accuracies)
plt.plot(x_axis,attack_success_rates)
plt.legend(['clean accuracy','attack success rate'])
plt.xlabel("fraction of pruned channels")
plt.ylabel("rate")
plt.title("Accuracy and attack success rate for validation dataset")

## BadNet + GoodNet
Now that we have our goodnet, we can combine it with the badnet and check predictions. 

In [ ]:
clean_test_filename = './data/cl/test.h5'
poisoned_test_filename = './data/bd/bd_test.h5'

In [ ]:
clean_x_test, clean_y_test = data_loader(clean_test_filename)
bad_x_test, bad_y_test = data_loader(poisoned_test_filename)

In [ ]:
# file paths 
drop_10 = './model_X=10.h5'
drop_4 = './model_X=2.h5'
drop_2 = './model_X=4.h5'
# load models
model_drop_10 = keras.models.load_model(drop_10)
model_drop_4 = keras.models.load_model(drop_4)
model_drop_2 = keras.models.load_model(drop_2)

In [ ]:
print(len(set(clean_y_test)))

The dataset has 1283 different classes, therefore we will keep N+1 as 1284

### Get Goodnet Predictions
Pass as arguments the BadNet, Pruned models with X={2,4,10}% and clean test data

In [ ]:
def predict_combined(model1,model2,data):
    #get predictions
    pred_model1= np.argmax(model1(data), axis=1)
    pred_model2= np.argmax(model2(data), axis=1)
    predictions = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        if pred_model1[i]==pred_model2[i]:
          predictions[i] = pred_model1[i]
        else:
          predictions[i] = 1284
    return predictions


### Get Goodnet Accuracy and Attack Success Rate 

In [ ]:
def get_accuracy_and_asr(goodnet):
    # to get accuracy predict on clean data
    goodnet_cl_pred = predict_combined(bad_model,goodnet,clean_x_test)
    accuracy = np.mean(np.equal(goodnet_cl_pred, clean_y_test))*100

    # to get asr predict on adverserial data
    goodnet_bd_pred = predict_combined(bad_model,goodnet,bad_x_test)
    asr = np.mean(np.equal(goodnet_bd_pred, bad_y_test))*100

    return accuracy,asr

### Test Goodness of GoodNets

In [ ]:
g_accuracy_10, g_asr_10 = get_accuracy_and_asr(model_drop_10)
print(f'Goodnet with 10% drop in accuracy has accuracy {g_accuracy_10} and attack success rate {g_asr_10}\n\n')

g_accuracy_4, g_asr_4 = get_accuracy_and_asr(model_drop_4)
print(f'Goodnet with 4% drop in accuracy has accuracy {g_accuracy_4} and attack success rate {g_asr_4}\n\n')

g_accuracy_2, g_asr_2 = get_accuracy_and_asr(model_drop_2)
print(f'Goodnet with 2% drop in accuracy has accuracy {g_accuracy_2} and attack success rate {g_asr_2}\n\n')

### Comparing Pruned Models and GoodNet Models
In this section we are conducting a thorough comparison of both the accuracy levels and attack success rates between our pruned models with a specified X% drop in accuracy and our consolidated GoodNets.

In [ ]:
def get_accuracy_and_asr_pruned(model):
    pruned_cl_pred = np.argmax(model.predict(clean_x_test), axis=1)
    accuracy = np.mean(np.equal(pruned_cl_pred, clean_y_test))*100

    pruned_bd_pred = np.argmax(model.predict(bad_x_test), axis=1)
    asr = np.mean(np.equal(pruned_bd_pred, bad_y_test))*100

    return accuracy,asr

In [ ]:
p_accuracy_10, p_asr_10 = get_accuracy_and_asr_pruned(model_drop_10)
print(f'Pruned network with 10% drop in accuracy has accuracy {p_accuracy_10} and attack success rate {p_asr_10}\n\n')

p_accuracy_4, p_asr_4 = get_accuracy_and_asr_pruned(model_drop_4)
print(f'Pruned network with 4% drop in accuracy has accuracy {p_accuracy_4} and attack success rate {p_asr_4}\n\n')

p_accuracy_2, p_asr_2 = get_accuracy_and_asr_pruned(model_drop_2)
print(f'Pruned network with 2% drop in accuracy has accuracy {p_accuracy_2} and attack success rate {p_asr_2}\n\n')

Therefore we see that our Goodnets perform comparable to the pruned models